# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze the tender for a project.
- Analyze the information about possible products, provided from different companies.
- Find the best combination of products to build the project in the tender, identifying the most compliant company as well.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- A tender for a project is parsed, such that its information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini, whereas a system prompt and a user prompt are written to explain the purposed o Gemini.

## Theoretical aspects
The current way to use Gemini makes use of the following properties of a LLM (Large Language Model) like Gemini:
|  **LLM property** | **Where it is used** | **How it is used** |
|:-----------------:|:--------------------:|:------------------:|
|     Reasoning     |          TBD         |         TBD        |
|       Memory      |          TBD         |         TBD        |
| Chain of Thoughts |          TBD         |         TBD        |
|                   |                      |                    |

Clean the working directory

In [1]:
! rm -r /kaggle/working/*

# Analyze the tenders

Locate the tenders - in PDF format.

In [2]:
# fetch the script to download content from GitHub
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py -P /kaggle/working/scripts

# add downloaded script to the Python path
import sys
sys.path.append('/kaggle/working/scripts')

--2024-11-22 15:39:21--  https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722 (1.7K) [text/plain]
Saving to: '/kaggle/working/scripts/github_downloader.py'

github_downloader.p 100%[===================>]   1.68K  --.-KB/s    in 0s      

2024-11-22 15:39:21 (26.0 MB/s) - '/kaggle/working/scripts/github_downloader.py' saved [1722/1722]



In [3]:
import github_downloader

github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder="/kaggle/working/tenders", extension="pdf")

/kaggle/working/tenders/tender_solar.pdf downloaded successfully.
/kaggle/working/tenders/tender_wind.pdf downloaded successfully.
All files downloaded.


In [4]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tenders') else True
if use_kaggle_input_tender:
    tenders_file_path = '/kaggle/input/tenders'
else:
    tenders_file_path = '/kaggle/working/tenders'

print(f"The folder {tenders_file_path} will be considered as containing the tenders")

The folder /kaggle/working/tenders will be considered as containing the tenders


Installing required Python packages to analyze the tender - in PDF format.

In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.6 MB/s eta 0:00:00


Extract information from the tenders.
The output will be a text.

In [6]:
import os
from PyPDF2 import PdfReader

tenders = [t for t in os.listdir(tenders_file_path) if t.endswith(".pdf")]
tenders_info = {}
for tender in tenders:
    print(f"Reading the tender {tender} ...")
    reader = PdfReader(os.path.join(tenders_file_path, tender))

    tenders_info[tender] = {}
    tenders_info[tender]["name"] = tender
    tenders_info[tender]["content"] = [page.extract_text() for page in reader.pages]

# temporary, considering the very first tender, only
tender_info = "".join(tenders_info[tenders[0]].get("content", {}))

Reading the tender tender_wind.pdf ...
Reading the tender tender_solar.pdf ...


# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [7]:
github_downloader.download_files_from_github_repo(folderName="", saveFolder="/kaggle/working/companies_info", extension="json")

/kaggle/working/companies_info/companies_info.json downloaded successfully.
All files downloaded.


Locate the JSON file containing the companies' information.

In [8]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info') else True
companies_json_name = 'companies_info.json'
if use_kaggle_input_companies:
    companies_info_file_path = os.path.join('/kaggle/input/companies-info', companies_json_name)
else:
    companies_info_file_path = os.path.join('/kaggle/working/companies_info', companies_json_name)

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

The file /kaggle/working/companies_info/companies_info.json will be used for the information regarding the companies


Define a small function to read the information about the companies - in JSON format.

In [9]:
import json

def read_companies_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

Load the companies' information by using the defined function.

In [10]:
companies_info = read_companies_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [11]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest')

chat = model.start_chat()

# example how to send prompts
# response = chat.send_message('Hi! What about competing on Kaggle?')
# print(response.text)

## Define the system and user prompts

In [12]:
system_prompt = "You are an experienced technical sales manager. You are given the content of websites of companies with their portfolio for products and solutions. Use the content of the websites to provide detailed technical answers based on tender requirements and user prompt."
# print(f"The system prompt is:\n{system_prompt}")

In [13]:
user_prompt = f"""

I want to build the whole plant as per tender technical requirements {tender_info}.

1. Analyse and identify all the technical requirements in the tender.

2. For company [SIEMENS] and [HITACHI], find the respective relevant products and solutions with respect to point 1. Do this one company at a time and store the results.

3. Calculate an affinity score in % of company [SIEMENS] and [HITACHI] based on the match of results in point 2.

SIEMENS: {companies_info["SIEMENS"]}
HITACHI: {companies_info["HITACHI"]}

4. Return a quick technical summary of the tender.

5. Return the technical details of the company with the highest affinity score and its score. Focus on the technical specifications mentioning the compliances with the tender. Report also the URL of the source where you found the informations and mention if there is some non compliant requirements from tender.

6. Return also the other affinity scores alone of the other companies. 

"""

# print(f"The user prompt is:\n{user_prompt}")

## Check how many tokens are needed to process the prompts

In [14]:
# Split the prompt into words (tokens)
tokens = user_prompt.split()

# Count the number of tokens
num_tokens = len(tokens)
print(f"Number of tokens (words): {num_tokens}")

Number of tokens (words): 420633


## Generate a response

In [15]:
want_a_response = False # disable if willing to save a notebook but the Gemini quota has been exhausted

if want_a_response:
    response = chat.send_message(system_prompt + user_prompt)
    print(response.text)

## Check how many tokens have been used so far

In [16]:
if want_a_response:
    usage_metadata = response.usage_metadata
    print(f"Token count: {usage_metadata.total_token_count}")